<img src='imgs/scylla_logo.png' width='750'/>

# docker

In [2]:
!docker run -d                   \
    -v ${PWD}/cassandra_vol:/ddl \
    -h cassandra                 \
    -p 7000:7000                 \
    -p 9042:9042                 \
    -p 9160:9160                 \
    --name scylla                \
    scylladb/scylla

50450c50af5176d88b46434f5f7711d4c16c51b87f9f46aa5be8ab9d3efc6621


# validate

In [3]:
!docker ps | grep scylla

50450c50af51        scylladb/scylla     "/docker-entrypoint.…"   18 seconds ago      Up 17 seconds       0.0.0.0:7000->7000/tcp, 0.0.0.0:9042->9042/tcp, 7001/tcp, 9180/tcp, 0.0.0.0:9160->9160/tcp, 10000/tcp   scylla


In [4]:
!docker exec -w /ddl -it scylla /usr/bin/cqlsh --version

cqlsh 5.0.1


# DDL

create keyspace

In [5]:
!docker exec -it scylla cqlsh -e \
"CREATE KEYSPACE elk_c WITH replication = {'class': 'NetworkTopologyStrategy', 'datacenter1': '2'} AND durable_writes = true"

create table

In [6]:
!docker exec -it scylla cqlsh -e \
"use elk_c;CREATE TABLE employee(id int PRIMARY KEY,name text,city text,salary varint,phone varint)"

In [7]:
!docker exec -it scylla cqlsh -e 'use elk_c;describe tables;' 


employee



In [8]:
!docker exec -it scylla cqlsh -e 'use elk_c; describe employee;' 


CREATE TABLE elk_c.employee (
    id int PRIMARY KEY,
    city text,
    name text,
    phone varint,
    salary varint
) WITH bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'ALL'}
    AND comment = ''
    AND compaction = {'class': 'SizeTieredCompactionStrategy'}
    AND compression = {'sstable_compression': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND crc_check_chance = 1.0
    AND dclocal_read_repair_chance = 0.1
    AND default_time_to_live = 0
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair_chance = 0.0
    AND speculative_retry = '99.0PERCENTILE';



# data

generate some random data

In [11]:
employees_csv = open('cassandra_vol/employees.csv', 'w')
# id int PRIMARY KEY,
# name text
# city text
# salary varint
# phone varint
employees_csv.write('1,Adam,NOLA,60,5551234\n')
employees_csv.write('2,Bob,LA,70,3331234\n')
employees_csv.write('3,Chuck,NYC,80,6661234\n')
employees_csv.write('4,Dave,BOS,90,9991234\n')
employees_csv.write('5,Earl,RAL,100,1111234\n')

employees_csv.flush()
employees_csv.close()
!cat cassandra_vol/employees.csv

1,Adam,NOLA,60,5551234
2,Bob,LA,70,3331234
3,Chuck,NYC,80,6661234
4,Dave,BOS,90,9991234
5,Earl,RAL,100,1111234


In [23]:
!docker exec -it scylla cqlsh -e "use elk_c;COPY employee FROM '/ddl/employees.csv'"

Using 9 child processes

Starting copy of elk_c.employee with columns [id, city, name, phone, salary].
Processed: 5 rows; Rate:      16 rows/s; Avg. rate:      16 rows/sProcessed: 5 rows; Rate:       8 rows/s; Avg. rate:      12 rows/s
5 rows imported from 1 files in 0.410 seconds (0 skipped).


# validate data

In [25]:
!time docker exec -it scylla \
cqlsh --connect-timeout 999 --request-timeout 999 \
-e "use elk_c;select count(*) from employee"


 count
-------
     5

(1 rows)

real	0m0.660s
user	0m0.034s
sys	0m0.017s


In [26]:
!time docker exec -it scylla cqlsh -e "use elk_c;select * from employee limit 5"


 id | city  | name | phone | salary
----+-------+------+-------+---------
  5 |  Earl |  RAL |   100 | 1111234
  1 |  Adam | NOLA |    60 | 5551234
  2 |   Bob |   LA |    70 | 3331234
  4 |  Dave |  BOS |    90 | 9991234
  3 | Chuck |  NYC |    80 | 6661234

(5 rows)

real	0m0.631s
user	0m0.038s
sys	0m0.020s


# stop & kill

In [27]:
!docker stop scylla && docker rm scylla
# or restart
# !docker restart cassandra

scylla
scylla
